# Implement shooting method to solve for time-path of variables

In [754]:
import numpy as np
from scipy.optimize import fsolve

In [755]:
# Define parameters
rho = 0.99
r = 0.05
a = 1
delta = 0.01
alpha = 0.7
gamma = 0.7
sigma = 0.7
params = (rho, r, a, delta, alpha, gamma, sigma)

# Initial conditions
K = 2
A = 3
mu = 0.2
p = 0.3
state = (K, A, mu, p)

In [756]:
# Define g(x) and g'(x) (labor market equilibrium tradeoff) function
def g(x):
    k = 5/4
    s = 1/2
    a = 1/(k**0.5 - s)
    y = k - (x/a + s)**2
    
    return y

def g_prime(x):
    k = 5/4
    s = 1/2
    a = 1/(k**0.5 - s)
    y = -2*(x/a + s)/a
    
    return y

In [757]:
# Define utility functions
def u(c, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    if c == 0:
        return -1e99
    
    return (c**(1-alpha))/(1-alpha)

def u_prime(c, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    if c == 0:
        return 1e99
    
    return c**(-alpha)

def v(l, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    if l == 0:
        return -1e99
    
    return (l**(1-gamma))/(1-gamma)

def v_prime(l, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    if l == 0:
        return 1e99
    
    return l**(-gamma)

def B(A, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    if A == 0:
        return -1e99
    
    return (A**(1-sigma))/(1-sigma)

def B_prime(A, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    
    if A == 0:
        return 1e99
    
    return B**(-sigma)

In [758]:
# Define functions to compute FOCs
# Consumption FOC
def c_foc(state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    
    c = mu**(-1/alpha)
    
    return c

# Leisure FOC
def l_rhs(x, state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    
    rhs = mu*K*g(x) + p*a*K*x
    
    return rhs

# Job choice FOC
def x_lhs(l, x, state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    
    h = 1-l
    lhs = h*(a*p*K + mu*K*g_prime(x))
    
    return lhs

In [759]:
# Define function that checks FOCs
def l_foc(l, x, state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    
    lhs = v_prime(l, params)
    rhs = l_rhs(x, state, params)
    diff = lhs - rhs
    
    return diff

def x_foc(l, x, state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    
    lhs = x_lhs(l, x, state, params) - 0
    
    return lhs

In [760]:
# Combine both FOC checkers within one function
def lx_foc(guess, state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    l, x = guess # Unpack guess vector
    
    res1 = l_foc(l, x, state, params) # Check FOC for l
    res2 = x_foc(l, x, state, params) # Check FOC for x
    
    error = np.array([res1, res2])
    
    return error

In [761]:
sol = fsolve(lx_foc, [2, 3], args = (state, params))

In [762]:
# Define a function that solves for c, l, and x for a given mu and p from FOCs
def foc_solve(state, params):
    rho, r, a, delta, alpha, gamma, sigma = params # Unpack parameters
    K, A, mu, p = state # Unpack state vector
    
    c = c_foc(state, params)
    
    # Try looking for interior solutions first
    [l, x] = fsolve(lx_foc, [2, 3], args = (state, params))
    
    # Check if solutions are interior
    if 0 < l < 1 and 0 < x < 1:
        print("Interior solution obtained")
        return np.array([l, x])
    
    # If not interior, try boundary solutions for x
    # Try x = 0
    x = 0
    [l] = fsolve(l_foc, [1], args = (x, state, params))
    # Check if l is valid
    if 0 < l < 1:
        # Check FOC for x holds
        check = x_lhs(l, x, state, params)
        if check <= 0:
            print("x = 0 is a valid boundary solution")
            return np.array([l, x])
        
    # If l is not valid, try x = 1
    x = 1
    [l] = fsolve(l_foc, [1], args = (x, state, params))
    if 0 < l < 1:
        # Check FOC for x holds
        check = x_lhs(l, x, state, params)
        if check >= 0:
            print("x = 1 is a valid boundary solution")
            return np.array([l, x])
    
    # If neither x = 0 nor x = 1 are valid, return the solution l = 1
    print("l = 1 is a valid boundary solution")
    return np.array([1, 0])

In [763]:
# Initial conditions
K = 0
A = 0
mu = 0
p = 0
state = (K, A, mu, p)

In [764]:
[l, x] = foc_solve(state, params)

ZeroDivisionError: 0.0 cannot be raised to a negative power

In [ ]:
l

In [ ]:
x